In [4]:
library(tidyverse)
library(data.table)
library(dplyr)

UBC_ISD <- fread("UBC-ISD.csv", encoding = "UTF-8")

ERROR: Error in fread("UBC-ISD.csv", encoding = "UTF-8"): Opened 6.814GB (7316020036 bytes) file ok but could not memory map it. This is a 64bit process. There is probably not enough contiguous virtual memory available.
